In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=f4ea6ac29fb56161d6b06ab3d8e786bf3d37ce649007a5d8c90efcc878146c7d
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


Debido a la dificultad que nos encontramos al intentar realizar un ETL Job desde glue sobre los archivos JSON en el S3, decidimos procesarlos antes de subirlos al S3 para poder sobrellevar esta dificultad, esto lo hicimos con el siguiente código que permite transformar los archivos JSON a archivos PARQUET que nos permite trabajar con ellos de manera más sencilla como los necesitamos.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pathlib import Path


spark = SparkSession.builder.appName("Procesar SIATA JSON").getOrCreate()


input_path = "./siata/raw/"
output_path = "./siata/processed/"


json_files = ["co", "no", "so2", "no2", "ozono", "pm1", "pm10", "pm25"]

for file_name in json_files:

    df = spark.read.json(f"{input_path}/Datos_SIATA_Aire_{file_name}.json")


    df = df.withColumn("datos", explode(col("datos")))

    df = df.select(
        col("latitud"),
        col("codigoSerial"),
        col("datos.variableConsulta"),
        col("datos.fecha"),
        col("datos.calidad"),
        col("datos.valor"),
        col("nombre"),
        col("nombreCorto"),
        col("longitud")
    )

    df.coalesce(1).write.mode("overwrite").parquet(f"{output_path}{file_name}_processed.parquet")

    print(f"Archivo {file_name} procesado y guardado en {output_path}")

spark.stop()


ModuleNotFoundError: No module named 'pyspark'